In [ ]:
# Disclaimer:
# This file is part of the undergraduate thesis of Mr. Efthymis Michalis.
# The thesis was developed under the supervision of Assistant Prof. Aggelos
# Pikrakis, in the Department of Informatics, School of ICT, University of
# Piraeus, Greece.

In [ ]:
import sys
sys.path.append('.')
from methods import img2points, bounding_box_draw

from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np
from skimage import feature
from skimage.transform import rescale, resize
from nntool.trainer import Trainer
import tensorflow as tf
import os

In [ ]:
clusters = DBSCAN(np.sqrt(2)+1e-2, 2)

height_clusters = DBSCAN(6, 6)


tf.compat.v1.disable_eager_execution()
model = Trainer('../topology.txt', '../weights')
model.SetSession()
model.Initialize_Vars()
labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I',
            'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [ ]:
# change file_idx
file_idx = 0
file_path = os.path.join("../../plates", os.listdir("../../plates")[file_idx])
img = plt.imread(file_path)

plt.imshow(img)
plt.show()

In [ ]:
per = (500*1000/img.size)**(1/2)
img = resize(img, (int(img.shape[0]*per), int(img.shape[1]*per)), anti_aliasing=True, anti_aliasing_sigma=0.2)
plt.imshow(img)

In [ ]:
img_gray = img[:, :, 0]*1/3 + img[:, :, 1]*1/3 + img[:, :, 2]*1/3
if np.max(img_gray) > 1:
    img_gray *= 1/255

## Edge detection

In [ ]:
cany_img = feature.canny(img_gray, 2, 0.3, 0.6)
plt.imshow(cany_img, cmap='gray')
plt.show()

## Segmentation based on clustering

In [ ]:
points = img2points(cany_img, 0)
clusters.fit(points)
points_labels = clusters.labels_

In [ ]:

unique_labels = set(points_labels)
corners = []
for un_point_label in unique_labels:
    mask_un_label = np.where(points_labels == un_point_label)
    class_points = points[mask_un_label]
    corners.append([np.min(class_points[:, 0]), np.min(class_points[:, 1]),
                    np.max(class_points[:, 0]), np.max(class_points[:, 1])
                    ])
corners = np.array(corners)

bounding_box_draw(img_gray, corners)

In [ ]:
plt.figure()
plt.plot(corners[:, 0], corners[:, 2], 'o', label='other objects')

height_clusters.fit(corners[:, [0,2]])
mask = np.where(height_clusters.labels_ != -1)
corners = corners[mask]

plt.xlabel(r'$PosY_{min}$', fontsize=16)
plt.ylabel(r'$PosY_{max}$', fontsize=16)
plt.plot(corners[:, 0], corners[:, 2], 'or', label='possible characters')
plt.title('Density = '+str(15)+', Radius = '+str(4), fontsize=16)
plt.legend(fontsize=16)

plt.figure()
bounding_box_draw(img_gray, corners)

## Test recognition

In [ ]:
label_obj = []
for i, corner in enumerate(corners):
    corner[[0,1]] += -1
    corner[[2,3]] += 1
    crop_img = img_gray[corner[0]:corner[2], corner[1]:corner[3]]
    label_obj.append([corner[1]])
    crop_img = 1 - crop_img
    crop_img = crop_img > 0.5
    dx = corner[2] - corner[0]
    dy = corner[3] - corner[1]
    avg = abs(dx-dy)/2
    panding0 = int(avg)+int(avg != int(avg))
    panding1 = int(avg)
    if dx > dy:
        crop_img = np.concatenate((np.zeros((dx, panding0)), crop_img, np.zeros((dx, panding1))), axis=1)
    elif dx < dy:
        crop_img = np.concatenate((np.zeros((panding0, dy)), crop_img, np.zeros((panding1, dy))), axis=0)

    scale_img = rescale(crop_img, (28-4)/crop_img.shape[0], anti_aliasing=True)
    scale_img = np.concatenate((np.zeros((28-4, 2)), scale_img, np.zeros((28-4, 2))), axis=1)
    scale_img = np.concatenate((np.zeros((2, 28)), scale_img, np.zeros((2, 28))), axis=0)

    confidence = model.Layers[-1].eval(feed_dict={
                     model.Layers[0]: scale_img.reshape(1, 28, 28, 1),
                     model.keep_prob: np.ones((model.DroupoutsProbabilitys.shape[0]))
            })[0]
    confidence_argmax = np.argmax(confidence)
    label = labels[confidence_argmax]
    confidence_max = np.max(confidence)
    plt.figure(figsize=(3,3))
    plt.title("Confidence:\n %.2f\nClass: %s" % (confidence_max, label))
    plt.axis('off')
    plt.imshow(scale_img, cmap='Greys_r')

    label_obj[-1].append(label)


pred_plate = ''
for label in sorted(label_obj, key=lambda chari: chari[0]):
    pred_plate += label[1]

In [ ]:
plt.title('Pred car plate: {}'.format(pred_plate))
plt.imshow(img)